# Model inferencing (fine-tuned Llama-2-7b-chat)

In [1]:
import os
import re
import torch
import pandas as pd
import warnings
from datasets import Dataset
from peft import PeftModel
from tqdm import tqdm
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
)

os.chdir("../../")   # current location is /notebooks/llama

In [2]:
pwd

'c:\\Users\\HERO\\OneDrive - connect.hku.hk\\!HKU Course\\FINA4350 Text Analytics and Natural Language Processing in Finance and Fintech\\proj1\\fina-nlp-project'

### Load and preprocess testing data

In [2]:
test_path = "data/processed/test_esg_shortened.csv"
df_test = pd.read_csv(test_path)
print(df_test.shape)

(138, 6)


In [3]:
df_test.head()

,symbol,year,quarter,transcript_esg,esg_score,esg_risk_level
0,ADP,2022,1,call reference nongaap financial measures beli...,14.0,Low
1,ADP,2022,1,call reference nongaap financial measures beli...,14.0,Low
2,ADSK,2022,1,addition replay call available autodeskcominve...,16.0,Negligible
3,ADSK,2022,3,addition replay call available autodeskcominve...,16.0,Negligible
4,ADSK,2022,4,addition replay call available autodeskcominve...,16.0,Negligible


In [4]:
def get_question_prompt(transcript_esg):
    system_msg = "<<SYS>>\n" \
        + "Your task is to predict the ESG risk level based on earning calls transcripts.\n" \
        + "The ESG Risk Ratings are categorized across five risk levels: negligible (0-10), low (10-20), medium (20-30), high (30-40), and severe (40+).\n" \
        + "The predicted risk level should always be one of the following: ['Negligible', 'Low', 'Medium', 'High', 'Severe'].\n" \
        + "When provided with an earning calls transcript, you should output the predicted ESG risk level along with the corresponding ESG score.\n" \
        + "Please format your answer as \"{score}, {label}\", where the score is the ESG score rounded to one decimal place, and the label is the predicted ESG risk level.\n" \
        + "Ensure that your predicted risk level is consistent with the predicted score.\n" \
        + "<</SYS>>\n\n"
    prompt = f"<s>[INST] {system_msg}###User: {transcript_esg}###You: [/INST]</s>"
    return prompt

df_test["prompt"] = df_test["transcript_esg"].apply(get_question_prompt)
df_test[["transcript_esg", "prompt"]].head()

,transcript_esg,prompt
0,call reference nongaap financial measures beli...,<s>[INST] <<SYS>>\nYour task is to predict the...
1,call reference nongaap financial measures beli...,<s>[INST] <<SYS>>\nYour task is to predict the...
2,addition replay call available autodeskcominve...,<s>[INST] <<SYS>>\nYour task is to predict the...
3,addition replay call available autodeskcominve...,<s>[INST] <<SYS>>\nYour task is to predict the...
4,addition replay call available autodeskcominve...,<s>[INST] <<SYS>>\nYour task is to predict the...


In [5]:
dataset_test = Dataset.from_pandas(df_test)
for entry in dataset_test:
    print(entry["prompt"])
    break

<s>[INST] <<SYS>>
Your task is to predict the ESG risk level based on earning calls transcripts.
The ESG Risk Ratings are categorized across five risk levels: negligible (0-10), low (10-20), medium (20-30), high (30-40), and severe (40+).
The predicted risk level should always be one of the following: ['Negligible', 'Low', 'Medium', 'High', 'Severe'].
When provided with an earning calls transcript, you should output the predicted ESG risk level along with the corresponding ESG score.
Please format your answer as "{score}, {label}", where the score is the ESG score rounded to one decimal place, and the label is the predicted ESG risk level.
Ensure that your predicted risk level is consistent with the predicted score.
<</SYS>>

###User: call reference nongaap financial measures believe useful investors excludes impact certain items description items along reconciliation nongaap measures comparable gaap measures found earnings release todays call also contain forwardlooking statements ref

### Load fine-tuned model

In [6]:
base_model = "meta-llama/Llama-2-7b-chat-hf"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [8]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [9]:
base_model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map="auto"
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
model_id = "model_240411_0952"
adapter_model = f"models/{model_id}"
model = PeftModel.from_pretrained(
    model=base_model,
    model_id=adapter_model
)
model = model.merge_and_unload()
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Lla

### Model inferencing demo

In [11]:
pipe = pipeline(
    task="text-generation", 
    model=model, 
    tokenizer=tokenizer, 
    max_length=1024,
    device_map="auto"
)

In [12]:
question = "I am really worried about my schoolwork. I think I can never finish them on time and is likely to fail all my exams this semester. What should I do?"
result = pipe(get_question_prompt(question))
print(result[0]['generated_text'])

<s>[INST] <<SYS>>
You are a helpful and honest psychologist.Respond to the following question from the user.Your answers should not include any critical, dangerous or illegal content.Your answers should be unbiased and encouraging.
<</SYS>>

###User: I am really worried about my schoolwork. I think I can never finish them on time and is likely to fail all my exams this semester. What should I do?###You: [/INST]  Thank you for reaching out for support! It's completely normal to feel overwhelmed with schoolwork, especially during exams season. Here are some suggestions that may help you manage your workload and reduce your stress levels:

1. Break down your tasks: Divide your assignments and exams into smaller, manageable chunks. Create a schedule for each task, including deadlines and study sessions. This will help you stay organized and focused.
2. Prioritize your tasks: Identify the most critical tasks that require your immediate attention. Focus on completing these tasks first, and t

### Model inferencing on test data

In [13]:
chunk_size = 4
num_chunks = df_test.shape[0] // chunk_size + int(df_test.shape[0] % chunk_size > 0)
results = []
with warnings.catch_warnings(category=UserWarning):
    warnings.filterwarnings("ignore")
    for i in tqdm(range(num_chunks)):
        result = pipe(df_test["prompt"].to_list()[i*chunk_size: (i+1)*chunk_size])
        results.extend(result)
print(results[0])

 32%|███▏      | 9/28 [10:16<20:43, 65.44s/it]--- Logging error ---
Traceback (most recent call last):
  File "/home/wk3user3/.conda/envs/apai-nlp/lib/python3.11/logging/__init__.py", line 1110, in emit
    msg = self.format(record)
          ^^^^^^^^^^^^^^^^^^^
  File "/home/wk3user3/.conda/envs/apai-nlp/lib/python3.11/logging/__init__.py", line 953, in format
    return fmt.format(record)
           ^^^^^^^^^^^^^^^^^^
  File "/home/wk3user3/.conda/envs/apai-nlp/lib/python3.11/logging/__init__.py", line 687, in format
    record.message = record.getMessage()
                     ^^^^^^^^^^^^^^^^^^^
  File "/home/wk3user3/.conda/envs/apai-nlp/lib/python3.11/logging/__init__.py", line 377, in getMessage
    msg = msg % self.args
          ~~~~^~~~~~~~~~~
TypeError: not all arguments converted during string formatting
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/wk3user3/.conda/envs/apai-nlp/lib/python3

[{'generated_text': "<s>[INST] <<SYS>>\nYou are a helpful and honest psychologist.Respond to the following question from the user.Your answers should not include any critical, dangerous or illegal content.Your answers should be unbiased and encouraging.\n<</SYS>>\n\n###User: I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac.    I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years.\n   I’ve never had counseling about any of this. Do I have too many issues to address in counseling?###You: [/INST]  Thank you for sharing your personal struggles with me. It takes a lot of courage to open up about such sensitive topics, and I want to acknowledge your bravery. It's important to recognize that you have been through a lot, and it's completely understandable that you may be feeling overwhelmed and unsure of where to turn.\n\nFirst

In [14]:
df_test["generatedText"] = [res[0]["generated_text"] for res in results]
df_test["generatedAnswerText"] = df_test["generatedText"].apply(
    lambda text: re.split(r"\[\/INST\]\s*", text, maxsplit=1)[1]
)
df_test.head()

,questionTitle,questionText,topic,answerText,upvotes,views,prompt,generatedText,generatedAnswerText
questionID,,,,,,,,,
0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,depression,It is very common for people to have multiple ...,3,1971,<s>[INST] <<SYS>>\nYou are a helpful and hones...,<s>[INST] <<SYS>>\nYou are a helpful and hones...,Thank you for sharing your personal struggles ...
2,I feel like my mother doesn't support me,My mother is combative with me when I say I do...,depression,Do you live with your mom and have constant in...,2,187,<s>[INST] <<SYS>>\nYou are a helpful and hones...,<s>[INST] <<SYS>>\nYou are a helpful and hones...,Thank you for reaching out for support. It's c...
11,Why am I experiencing dfficulty maintaining an...,A few years ago I was making love to my wife w...,depression,When I'm working with men with this type of si...,1,194,<s>[INST] <<SYS>>\nYou are a helpful and hones...,<s>[INST] <<SYS>>\nYou are a helpful and hones...,Thank you for reaching out for support. It tak...
14,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,depression,"Hello, and thank you for your question and see...",2,5423,<s>[INST] <<SYS>>\nYou are a helpful and hones...,<s>[INST] <<SYS>>\nYou are a helpful and hones...,Thank you for sharing your struggles with me. ...
16,How can I stop treating people badly?,I feel like I'm so alone. I treat people horri...,depression,Give yourself a little more credit for self-ob...,1,1029,<s>[INST] <<SYS>>\nYou are a helpful and hones...,<s>[INST] <<SYS>>\nYou are a helpful and hones...,Thank you for sharing your feelings with me. I...


In [15]:
# df_test[
#     ["topic", "questionText", "answerText", "generatedAnswerText"]
# ].to_csv(f"response/test-response-{model_id}.csv")